In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/Machine learning

/content/drive/My Drive/Machine learning


In [0]:
from pathlib import Path

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

pd.set_option("display.max_columns", 100)

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 

In [0]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

In [6]:
!pip install catboost

In [0]:
from catboost import CatBoostClassifier

In [0]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) 

In [0]:
features_df = pd.read_csv(
    "train.csv", 
    index_col="tripid"
)


In [0]:
features_df.drop( features_df[ features_df['pick_lat'] < 5.9 ].index , inplace=True)
features_df.drop( features_df[ features_df['pick_lat'] > 9.82 ].index , inplace=True)	
features_df.drop( features_df[ features_df['pick_lon'] < 79 ].index , inplace=True)
features_df.drop( features_df[ features_df['pick_lon'] > 82 ].index , inplace=True)
features_df.drop( features_df[ features_df['drop_lat'] < 5.9 ].index , inplace=True)
features_df.drop( features_df[ features_df['drop_lat'] > 9.82 ].index , inplace=True)	
features_df.drop( features_df[ features_df['drop_lon'] < 79 ].index , inplace=True)
features_df.drop( features_df[ features_df['drop_lon'] > 82 ].index , inplace=True)

In [0]:
features_df = features_df.drop_duplicates()

In [12]:
features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct


In [0]:
features_df["distance"] =  distance(features_df["pick_lat"],features_df["pick_lon"],features_df["drop_lat"],features_df["drop_lon"])

In [0]:
features_df["pickup_time"] = pd.to_datetime(features_df["pickup_time"],errors = "coerce")
features_df["drop_time"] = pd.to_datetime(features_df["drop_time"],errors = "coerce")
features_df['duration'] = features_df['duration'].fillna((features_df['drop_time'] - features_df['pickup_time']).astype('timedelta64[s]'))

In [0]:
features_df['additional_fare'] = features_df['additional_fare'].fillna(features_df['additional_fare'].mode().iloc[0])

In [0]:
features_df_correct = features_df[features_df.label=='correct']
features_df_incorrect = features_df[features_df.label=='incorrect']

In [17]:
features_df_correct['meter_waiting'] = features_df_correct['meter_waiting'].fillna(features_df_correct['meter_waiting'].mean())
features_df_incorrect['meter_waiting'] = features_df_incorrect['meter_waiting'].fillna(features_df_incorrect['meter_waiting'].mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
features_df_correct['meter_waiting_fare'] = features_df_correct['meter_waiting_fare'].fillna(features_df_correct['meter_waiting_fare'].mean())
features_df_incorrect['meter_waiting_fare'] = features_df_incorrect['meter_waiting_fare'].fillna(features_df_incorrect['meter_waiting_fare'].mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
features_df_correct['fare'] = features_df_correct['fare'].fillna((92.33)*features_df_correct['distance']+features_df_correct['additional_fare']+features_df_correct['meter_waiting_fare'])
features_df_incorrect['fare'] = features_df_incorrect['fare'].fillna((221)*features_df_incorrect['distance']+features_df_incorrect['additional_fare']+features_df_incorrect['meter_waiting_fare'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
features_df = pd.concat([features_df_correct,features_df_incorrect],sort=True)

In [0]:
y= features_df["label"]

In [0]:
y =y.replace(to_replace="correct",value=1)
y =y.replace(to_replace="incorrect",value=0)

In [23]:
y[189157607]


0

In [0]:
X = features_df.drop(columns=["label"], axis=1)

In [0]:
test_features_df = pd.read_csv("test.csv", 
                               index_col="tripid")

In [26]:
test_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.44860,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.00000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.65880,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.00000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.36920,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47
...,...,...,...,...,...,...,...,...,...,...,...,...
222856243,10.5,1723,429,24.83332,3,3/16/2020 21:28,3/16/2020 21:56,6.85103,79.9567,6.85588,79.9214,388.48
222857785,10.5,1378,80,0.00000,125,3/16/2020 21:59,3/16/2020 22:22,6.91293,79.9656,6.92112,79.8980,379.85
222858416,10.5,418,56,3.28440,93,3/16/2020 22:02,3/16/2020 22:09,6.85718,79.9081,6.83868,79.9083,112.79


In [27]:
X

,additional_fare,distance,drop_lat,drop_lon,drop_time,duration,fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,pickup_time
tripid,,,,,,,,,,,,,
189123628,10.5,3.164501,6.90330,79.8783,2019-11-01 00:34:00,834.0,270.320000,56.000000,0.000000,64.0,6.86252,79.8993,2019-11-01 00:20:00
189125358,10.5,1.968540,6.91373,79.8923,2019-11-01 01:09:00,791.0,197.850000,47.000000,0.000000,134.0,6.88589,79.8984,2019-11-01 00:56:00
189125719,10.5,3.917991,6.93669,79.9146,2019-11-01 01:26:00,1087.0,301.640000,80.000000,0.000000,61.0,6.90839,79.8651,2019-11-01 01:08:00
189127273,10.5,0.535588,6.92748,79.8971,2019-11-01 02:37:00,598.0,82.300000,271.000000,15.663800,68.0,6.92570,79.8895,2019-11-01 02:27:00
189128020,10.5,5.062797,6.84478,79.9290,2019-11-01 03:51:00,1020.0,358.390000,356.675539,16.910844,NaN,6.87441,79.8615,2019-11-01 03:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213674502,10.5,8.475084,6.97458,79.9380,2020-01-31 17:59:00,4380.0,2068.533683,3379.965991,185.040038,NaN,6.87969,79.8597,2020-01-31 16:46:00
213689462,40.5,6.023865,6.83700,79.8947,2020-01-31 18:34:00,4310.0,902.020000,1314.000000,75.949200,469.0,6.91819,79.8627,2020-01-31 17:22:00
213703973,40.5,0.711581,7.30808,80.6399,2020-01-31 17:54:00,913.0,534.000000,445.000000,0.000000,47.0,7.29842,80.6363,2020-01-31 17:39:00


In [0]:
#X = X.dropna()

In [29]:
X

,additional_fare,distance,drop_lat,drop_lon,drop_time,duration,fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,pickup_time
tripid,,,,,,,,,,,,,
189123628,10.5,3.164501,6.90330,79.8783,2019-11-01 00:34:00,834.0,270.320000,56.000000,0.000000,64.0,6.86252,79.8993,2019-11-01 00:20:00
189125358,10.5,1.968540,6.91373,79.8923,2019-11-01 01:09:00,791.0,197.850000,47.000000,0.000000,134.0,6.88589,79.8984,2019-11-01 00:56:00
189125719,10.5,3.917991,6.93669,79.9146,2019-11-01 01:26:00,1087.0,301.640000,80.000000,0.000000,61.0,6.90839,79.8651,2019-11-01 01:08:00
189127273,10.5,0.535588,6.92748,79.8971,2019-11-01 02:37:00,598.0,82.300000,271.000000,15.663800,68.0,6.92570,79.8895,2019-11-01 02:27:00
189128020,10.5,5.062797,6.84478,79.9290,2019-11-01 03:51:00,1020.0,358.390000,356.675539,16.910844,NaN,6.87441,79.8615,2019-11-01 03:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213674502,10.5,8.475084,6.97458,79.9380,2020-01-31 17:59:00,4380.0,2068.533683,3379.965991,185.040038,NaN,6.87969,79.8597,2020-01-31 16:46:00
213689462,40.5,6.023865,6.83700,79.8947,2020-01-31 18:34:00,4310.0,902.020000,1314.000000,75.949200,469.0,6.91819,79.8627,2020-01-31 17:22:00
213703973,40.5,0.711581,7.30808,80.6399,2020-01-31 17:54:00,913.0,534.000000,445.000000,0.000000,47.0,7.29842,80.6363,2020-01-31 17:39:00


In [0]:
X = X.fillna(X.mean())
#features_df = features_df.fillna(features_df.mean())


In [0]:
test_features_df = test_features_df.fillna(test_features_df.mean())

In [0]:
X["pickup_time"] = pd.to_datetime(X["pickup_time"],errors = "coerce")

In [0]:
X["drop_time"] = pd.to_datetime(X["drop_time"],errors = "coerce")

In [0]:
X["pickup_time_hour"] = X["pickup_time"].dt.hour

In [0]:
X["pickup_time_minute"] = X["pickup_time"].dt.minute

In [0]:
X["drop_time_hour"] =X["drop_time"].dt.hour

In [0]:
X["drop_time_minute"] =X["drop_time"].dt.minute

In [0]:
X["pickup_time_day"] = X["pickup_time"].dt.day
# X["pickup_time_month"] = X["pickup_time"].dt.month
X["drop_time_day"] = X["drop_time"].dt.day
# X["drop_time_month"] = X["drop_time"].dt.month
# X["pick_up_year"] = X["drop_time"].dt.year
# X["drop_year"] = X["drop_time"].dt.year

In [0]:
# X["distance"] = (X["pick_lat"]-X["drop_lat"])**2 + (X["pick_lon"]-X["drop_lon"])**2
# X["distance"] =  distance(X["pick_lat"],X["pick_lon"],X["drop_lat"],X["drop_lon"])

In [42]:
X.dtypes

additional_fare                     float64
distance                            float64
drop_lat                            float64
drop_lon                            float64
drop_time                    datetime64[ns]
duration                            float64
fare                                float64
meter_waiting                       float64
meter_waiting_fare                  float64
meter_waiting_till_pickup           float64
pick_lat                            float64
pick_lon                            float64
pickup_time                  datetime64[ns]
pickup_time_hour                      int64
pickup_time_minute                    int64
drop_time_hour                        int64
drop_time_minute                      int64
pickup_time_day                       int64
drop_time_day                         int64
dtype: object

In [0]:
X["effective_time"] = X["duration"]-X["meter_waiting"]


In [0]:
X.loc[X['fare'] <= 2000, 'fare_outlier'] = 1
X.loc[X['fare'] > 2000, 'fare_outlier'] = 0

In [0]:
#X["effective_time"] = (X["effective_time"] -X["effective_time"].mean() )/X["effective_time"].std() 

In [0]:
# X["effective_fare"] = X["fare"]+X["additional_fare"]

In [0]:
#X["effective_fare"] = (X["effective_fare"] -X["effective_fare"].mean() )/X["effective_fare"].std() 

In [0]:
# X["mean_fare_for_unit_length"] =X["fare"]/ X["distance"]

In [0]:
#X["mean_fare_for_unit_length"] = (X["mean_fare_for_unit_length"] -X["mean_fare_for_unit_length"].mean() )/X["mean_fare_for_unit_length"].std() 

In [0]:
# X["duration_fare"] = X["duration"]*X["fare"]

In [0]:
#X["duration_fare"] = (X["duration_fare"] -X["duration_fare"].mean() )/X["duration_fare"].std() 

In [0]:
# X = X.drop(columns=["pickup_time","drop_time"],axis=1)

In [59]:
X

,additional_fare,distance,drop_lat,drop_lon,drop_time,duration,fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,pickup_time,pickup_time_hour,pickup_time_minute,drop_time_hour,drop_time_minute,pickup_time_day,drop_time_day,effective_time,fare_outlier
tripid,,,,,,,,,,,,,,,,,,,,,
189123628,10.5,3.164501,6.90330,79.8783,2019-11-01 00:34:00,834.0,270.320000,56.000000,0.000000,64.000000,6.86252,79.8993,2019-11-01 00:20:00,0,20,0,34,1,1,778.000000,1.0
189125358,10.5,1.968540,6.91373,79.8923,2019-11-01 01:09:00,791.0,197.850000,47.000000,0.000000,134.000000,6.88589,79.8984,2019-11-01 00:56:00,0,56,1,9,1,1,744.000000,1.0
189125719,10.5,3.917991,6.93669,79.9146,2019-11-01 01:26:00,1087.0,301.640000,80.000000,0.000000,61.000000,6.90839,79.8651,2019-11-01 01:08:00,1,8,1,26,1,1,1007.000000,1.0
189127273,10.5,0.535588,6.92748,79.8971,2019-11-01 02:37:00,598.0,82.300000,271.000000,15.663800,68.000000,6.92570,79.8895,2019-11-01 02:27:00,2,27,2,37,1,1,327.000000,1.0
189128020,10.5,5.062797,6.84478,79.9290,2019-11-01 03:51:00,1020.0,358.390000,356.675539,16.910844,112.469299,6.87441,79.8615,2019-11-01 03:34:00,3,34,3,51,1,1,663.324461,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213674502,10.5,8.475084,6.97458,79.9380,2020-01-31 17:59:00,4380.0,2068.533683,3379.965991,185.040038,112.469299,6.87969,79.8597,2020-01-31 16:46:00,16,46,17,59,31,31,1000.034009,0.0
213689462,40.5,6.023865,6.83700,79.8947,2020-01-31 18:34:00,4310.0,902.020000,1314.000000,75.949200,469.000000,6.91819,79.8627,2020-01-31 17:22:00,17,22,18,34,31,31,2996.000000,1.0
213703973,40.5,0.711581,7.30808,80.6399,2020-01-31 17:54:00,913.0,534.000000,445.000000,0.000000,47.000000,7.29842,80.6363,2020-01-31 17:39:00,17,39,17,54,31,31,468.000000,1.0


In [0]:
# X2 = X.values 
# x2_scaled = min_max_scaler.fit_transform(X2)
# X = pd.DataFrame(x2_scaled)

In [0]:
test_features_df["pickup_time"] = pd.to_datetime(test_features_df["pickup_time"],errors = "coerce")
test_features_df["drop_time"] = pd.to_datetime(test_features_df["drop_time"],errors = "coerce")
test_features_df["pickup_time_hour"] = test_features_df["pickup_time"].dt.hour
test_features_df["pickup_time_minute"] = test_features_df["pickup_time"].dt.minute
test_features_df["drop_time_hour"] =test_features_df["drop_time"].dt.hour
test_features_df["drop_time_minute"] =test_features_df["drop_time"].dt.minute
test_features_df["effective_time"] = test_features_df["duration"]-test_features_df["meter_waiting"]
test_features_df["pickup_time_day"] = test_features_df["pickup_time"].dt.day
# test_features_df["pickup_time_month"] = test_features_df["pickup_time"].dt.month
test_features_df["drop_time_day"] = test_features_df["drop_time"].dt.day
# test_features_df["drop_time_month"] = test_features_df["drop_time"].dt.month
# test_features_df["pick_up_year"] = test_features_df["drop_time"].dt.year
# test_features_df["drop_year"] = test_features_df["drop_time"].dt.year
# test_features_df["distance"] =(test_features_df["pick_lat"]-test_features_df["drop_lat"])**2 + (test_features_df["pick_lon"]-test_features_df["drop_lon"])**2
test_features_df["distance"] =  distance(test_features_df["pick_lat"],test_features_df["pick_lon"],test_features_df["drop_lat"],test_features_df["drop_lon"])

# test_features_df["effective_fare"] = test_features_df["fare"]+test_features_df["additional_fare"]
# test_features_df["mean_fare_for_unit_length"] = test_features_df["fare"]/test_features_df["distance"]
# test_features_df["duration_fare"] = test_features_df["duration"]*test_features_df["fare"]

In [0]:
# test_features_df["effective_time"] = (test_features_df["effective_time"] -test_features_df["effective_time"].mean() )/test_features_df["effective_time"].std() 
# test_features_df["effective_fare"] = (test_features_df["effective_fare"] -test_features_df["effective_fare"].mean() )/test_features_df["effective_fare"].std() 
# test_features_df["mean_fare_for_unit_length"] = (test_features_df["mean_fare_for_unit_length"] -test_features_df["mean_fare_for_unit_length"].mean() )/test_features_df["mean_fare_for_unit_length"].std() 
# test_features_df["duration_fare"] = (test_features_df["duration_fare"] -test_features_df["duration_fare"].mean() )/test_features_df["duration_fare"].std() 


In [0]:
# test_features_df = test_features_df.drop(columns=["pickup_time","drop_time"],axis=1)
test_features_df.loc[test_features_df['fare'] <= 2000, 'fare_outlier'] = 1
test_features_df.loc[test_features_df['fare'] > 2000, 'fare_outlier'] = 0

In [58]:
# X2 = test_features_df.values 
# x2_scaled = min_max_scaler.fit_transform(X2)
# test_features_df = pd.DataFrame(x2_scaled)
test_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_hour,pickup_time_minute,drop_time_hour,drop_time_minute,effective_time,pickup_time_day,drop_time_day,distance,fare_outlier
tripid,,,,,,,,,,,,,,,,,,,,,
213284604,10.5,924,42,2.44860,148,2020-02-01 00:38:00,2020-02-01 00:53:00,6.83454,79.8750,6.77490,79.8840,289.27,0,38,0,53,882,1,1,4.166730,1.0
213286352,10.5,4249,20,0.00000,91,2020-02-01 01:02:00,2020-02-01 02:13:00,6.91168,79.8723,6.55091,79.9706,1912.70,1,2,2,13,4229,1,1,25.823263,1.0
213293973,10.5,1552,255,2.65880,23,2020-02-01 05:02:00,2020-02-01 05:28:00,6.92145,79.8478,6.90539,79.8989,394.00,5,2,5,28,1297,1,1,3.676453,1.0
213294622,10.5,462,16,0.00000,198,2020-02-01 05:30:00,2020-02-01 05:38:00,6.77433,79.9416,6.80401,79.9407,154.32,5,30,5,38,446,1,1,2.051619,1.0
213298687,10.5,814,392,12.36920,69,2020-02-01 07:00:00,2020-02-01 07:14:00,6.97968,79.9130,6.98875,79.8914,147.47,7,0,7,14,422,1,1,1.608445,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222856243,10.5,1723,429,24.83332,3,2020-03-16 21:28:00,2020-03-16 21:56:00,6.85103,79.9567,6.85588,79.9214,388.48,21,28,21,56,1294,16,16,2.444643,1.0
222857785,10.5,1378,80,0.00000,125,2020-03-16 21:59:00,2020-03-16 22:22:00,6.91293,79.9656,6.92112,79.8980,379.85,21,59,22,22,1298,16,16,4.671116,1.0
222858416,10.5,418,56,3.28440,93,2020-03-16 22:02:00,2020-03-16 22:09:00,6.85718,79.9081,6.83868,79.9083,112.79,22,2,22,9,362,16,16,1.278300,1.0


In [0]:
categorical_var1 = np.where(test_features_df.dtypes != np.float )[0]

In [61]:
categorical_var1

array([ 1,  2,  4,  5,  6, 12, 13, 14, 15, 16, 17, 18])

In [62]:
X.dtypes

additional_fare                     float64
distance                            float64
drop_lat                            float64
drop_lon                            float64
drop_time                    datetime64[ns]
duration                            float64
fare                                float64
meter_waiting                       float64
meter_waiting_fare                  float64
meter_waiting_till_pickup           float64
pick_lat                            float64
pick_lon                            float64
pickup_time                  datetime64[ns]
pickup_time_hour                      int64
pickup_time_minute                    int64
drop_time_hour                        int64
drop_time_minute                      int64
pickup_time_day                       int64
drop_time_day                         int64
effective_time                      float64
fare_outlier                        float64
dtype: object

In [0]:
categorical_var = np.where(X.dtypes != np.float )[0]

In [64]:
categorical_var

array([ 4, 12, 13, 14, 15, 16, 17, 18])

In [0]:
model = CatBoostClassifier(iterations=500000)

In [66]:
model.fit(X,y,plot=False)

Streaming output truncated to the last 5000 lines.
495001:	learn: 0.0496713	total: 2h 3m 54s	remaining: 1m 15s
495002:	learn: 0.0496712	total: 2h 3m 54s	remaining: 1m 15s
495003:	learn: 0.0496711	total: 2h 3m 54s	remaining: 1m 15s
495004:	learn: 0.0496711	total: 2h 3m 54s	remaining: 1m 15s
495005:	learn: 0.0496710	total: 2h 3m 54s	remaining: 1m 15s
495006:	learn: 0.0496709	total: 2h 3m 54s	remaining: 1m 14s
495007:	learn: 0.0496708	total: 2h 3m 54s	remaining: 1m 14s
495008:	learn: 0.0496708	total: 2h 3m 54s	remaining: 1m 14s
495009:	learn: 0.0496707	total: 2h 3m 54s	remaining: 1m 14s
495010:	learn: 0.0496707	total: 2h 3m 54s	remaining: 1m 14s
495011:	learn: 0.0496706	total: 2h 3m 54s	remaining: 1m 14s
495012:	learn: 0.0496705	total: 2h 3m 54s	remaining: 1m 14s
495013:	learn: 0.0496704	total: 2h 3m 54s	remaining: 1m 14s
495014:	learn: 0.0496704	total: 2h 3m 54s	remaining: 1m 14s
495015:	learn: 0.0496703	total: 2h 3m 54s	remaining: 1m 14s
495016:	learn: 0.0496702	total: 2h 3m 54s	remaini

In [0]:
prediction = model.predict(test_features_df)

In [68]:
prediction

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
submission_df = pd.read_csv("sample_submission.csv", 
                            index_col="tripid")

In [70]:
X.head()


,additional_fare,distance,drop_lat,drop_lon,drop_time,duration,fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,pickup_time,pickup_time_hour,pickup_time_minute,drop_time_hour,drop_time_minute,pickup_time_day,drop_time_day,effective_time,fare_outlier
tripid,,,,,,,,,,,,,,,,,,,,,
189123628,10.5,3.164501,6.90330,79.8783,2019-11-01 00:34:00,834.0,270.32,56.000000,0.000000,64.000000,6.86252,79.8993,2019-11-01 00:20:00,0,20,0,34,1,1,778.000000,1.0
189125358,10.5,1.968540,6.91373,79.8923,2019-11-01 01:09:00,791.0,197.85,47.000000,0.000000,134.000000,6.88589,79.8984,2019-11-01 00:56:00,0,56,1,9,1,1,744.000000,1.0
189125719,10.5,3.917991,6.93669,79.9146,2019-11-01 01:26:00,1087.0,301.64,80.000000,0.000000,61.000000,6.90839,79.8651,2019-11-01 01:08:00,1,8,1,26,1,1,1007.000000,1.0
189127273,10.5,0.535588,6.92748,79.8971,2019-11-01 02:37:00,598.0,82.30,271.000000,15.663800,68.000000,6.92570,79.8895,2019-11-01 02:27:00,2,27,2,37,1,1,327.000000,1.0
189128020,10.5,5.062797,6.84478,79.9290,2019-11-01 03:51:00,1020.0,358.39,356.675539,16.910844,112.469299,6.87441,79.8615,2019-11-01 03:34:00,3,34,3,51,1,1,663.324461,1.0


In [0]:
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)


In [0]:
submission_df["prediction"] = prediction[:]

In [73]:
submission_df.head()

,prediction
tripid,
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1


In [0]:
submission_df.to_csv('Multi_Higher_dimensional_Gamma_Mission.csv', index=True)

In [0]:

# model.save_model("model_trained_features_added_month_dropped_cbm", format="cbm")


In [0]:
cd /content/drive/My Drive/Machine learning

/content/drive/My Drive/Machine learning


In [0]:
from xgboost import XGBClassifier

In [0]:
model1 = XGBClassifier()

In [0]:
model1.fit(X, y)

ValueError: ignored

In [0]:
y_pred = model.predict(test_features_df)

In [0]:
X.shape

In [0]:
test_features_df.shape
